In [199]:
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
from uszipcode import ZipcodeSearchEngine
from geopy.geocoders import Nominatim

import pandas as pd
import seaborn.apionly as sns
from datetime import date, datetime
from haversine import haversine

# statistics package
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy import stats

# packages for mapping
from mpl_toolkits.basemap import Basemap

# packages for interactive graphs
from ipywidgets import widgets, interact
from IPython.display import display
from copy import deepcopy as copy
import time
%matplotlib inline

In [28]:
historical_data = pd.read_csv('train.csv')

In [29]:
historical_data.head()

,Unnamed: 0,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,...,pickup_latitude,dropoff_longitude,dropoff_latitude,fare_amount,payment_type,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
0,0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,1,N,2013-01-01 15:11:48,2013-01-01 15:18:10,4,382,...,40.757977,-73.989838,40.751171,6.5,CSH,0.0,0.5,0.0,0.0,7.0
1,1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-06 00:18:35,2013-01-06 00:22:54,1,259,...,40.731781,-73.994499,40.750660,6.0,CSH,0.5,0.5,0.0,0.0,7.0
2,2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-05 18:49:41,2013-01-05 18:54:23,1,282,...,40.737770,-74.009834,40.726002,5.5,CSH,1.0,0.5,0.0,0.0,7.0
3,3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:54:15,2013-01-07 23:58:20,2,244,...,40.759945,-73.984734,40.759388,5.0,CSH,0.5,0.5,0.0,0.0,6.0
4,4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:25:03,2013-01-07 23:34:24,1,560,...,40.748528,-74.002586,40.747868,9.5,CSH,0.5,0.5,0.0,0.0,10.5


## Data Preprocessing

In [33]:
process_train_data = copy(historical_data)

## Train Data

In [39]:
train_data = copy(process_train_data.loc[0:100000,])
del train_data['Unnamed: 0']
train_data.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,mta_tax,tip_amount,tolls_amount,total_amount,pickup_day,pickup_weekday,pickup_hour,pickup_minute,pickup_time,dropoff_hour
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,1,N,2013-01-01 15:11:48,2013-01-01 15:18:10,4,382,1.0,...,0.5,0.0,0.0,7.0,1,1,15,11,15.183333,15
1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-06 00:18:35,2013-01-06 00:22:54,1,259,1.5,...,0.5,0.0,0.0,7.0,6,6,0,18,0.300000,0
2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-05 18:49:41,2013-01-05 18:54:23,1,282,1.1,...,0.5,0.0,0.0,7.0,5,5,18,49,18.816667,18
3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:54:15,2013-01-07 23:58:20,2,244,0.7,...,0.5,0.0,0.0,6.0,7,0,23,54,23.900000,23
4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:25:03,2013-01-07 23:34:24,1,560,2.1,...,0.5,0.0,0.0,10.5,7,0,23,25,23.416667,23


In [40]:
train_data["pickup_day"] = train_data["pickup_datetime"].apply(lambda x: x.day)
train_data["pickup_weekday"] = train_data["pickup_datetime"].apply(lambda x: x.weekday())
train_data["pickup_hour"] = train_data["pickup_datetime"].apply(lambda x: x.hour)
train_data["pickup_minute"] = train_data["pickup_datetime"].apply(lambda x: x.minute)
train_data["pickup_time"] = train_data["pickup_hour"] + (df["pickup_minute"] / 60)
train_data["dropoff_hour"] = train_data["dropoff_datetime"].apply(lambda x: x.hour)

In [45]:
train_data['pickup_date'] = [date.date() for date in train_data['pickup_datetime']]
train_data['dropoff_date'] = [date.date() for date in train_data['dropoff_datetime']]

### The distance is calculated in kilometers

In [215]:
def distance(lat1, lon1, lat2, lon2):
    """calculates the Manhattan distance between 2 points
        using their coordinates
    
    Parameters
    ----------
    lat1: float
        latitude of first point
        
    lon1: float
        longitude of first point
        
    lat2: float
        latitude of second point
    
    lon2: float
        longitude of second point
        
    Returns
    -------
    d: float
        The Manhattan distance between the two points in kilometers
        
    """
    
    d = haversine((lat1, lon1), (lat2, lon1)) + haversine((lat2, lon1), (lat2, lon2))
    return d

In [42]:
train_data["distance"] = train_data.apply(lambda row: distance(row["pickup_latitude"], 
                                               row["pickup_longitude"], 
                                               row["dropoff_latitude"], 
                                               row["dropoff_longitude"]), axis=1)

### The speed is calculated in km/h

In [43]:
train_data["speed"] = train_data["distance"] / (train_data["trip_time_in_secs"] / 3600)

In [55]:
pickup_datetime = train_data['pickup_datetime']
day_interval = []
for i in range(pickup_datetime.size):
    hour = pickup_datetime[i].hour
    if hour>=6 and hour<12:
        day_interval.append(0)
    elif hour>=12 and hour<18:
        day_interval.append(1)
    elif hour>=18 and hour<24:
        day_interval.append(2)
    else:
        day_interval.append(3)
train_data['day_interval'] = day_interval

In [237]:
def add_waiting_time_and_penalty(train_data):
    sorted_train_data = copy(train_data.sort_values(by=['hack_license', 'pickup_datetime'], ascending=True))
    sorted_train_data['waiting_time'] = 0.0
    sorted_train_data['waiting_penalty'] = 0.0
    previous_license = None
    previous_drpoff_datatime = 0
    previous_date = None
    previous_row = None
    wait_time_list = []
    wait_penalty_list = []
    average_speed = sorted_train_data['speed'].mean()
    for index,row in sorted_train_data.T.iteritems():
        if row['hack_license'] == previous_license and row['pickup_date'] == previous_date:
            diff =  row['pickup_datetime'] - previous_row['dropoff_datetime']
            _distance = distance(row["pickup_latitude"], row["pickup_longitude"], previous_row["dropoff_latitude"], previous_row["dropoff_longitude"])
            speed = row['speed']
            if(speed <= 0):
                speed = average_speed
            time_take_to_reach = (_distance/speed)*3600
            if (time_take_to_reach>diff.total_seconds()):
                wait_time_list.append(0.0)
                wait_penalty_list.append(0.0)
            else:
                wait_time_list.append(diff.total_seconds()-time_take_to_reach)
                wait_penalty_list.append((diff.total_seconds()-time_take_to_reach)/3600*5)
        else:
            wait_time_list.append(0.0)
            previous_license = row['hack_license'] 
            previous_date = row['pickup_date']
            previous_row = row
            wait_penalty_list.append(0.0)
    sorted_train_data['waiting_time'] = wait_time_list
    sorted_train_data['waiting_penalty'] = wait_penalty_list
    return sorted_train_data

In [238]:
sorted_train_data = add_waiting_time_and_penalty(train_data)
sorted_train_data.head()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,...,pickup_time,dropoff_hour,distance,speed,pickup_date,dropoff_date,time_inteval,day_interval,waiting_time,waiting_penalty
5708,F4A9B95166FF93094F5B6A98D2D41B45,001C8AAB90AEE49F36FCAA7B4136C81A,VTS,2,NaN,2013-01-03 05:54:00,2013-01-03 05:55:00,1,60,0.00,...,5.900000,5,12723.128217,763387.692992,2013-01-03,2013-01-03,Night,3,0.000000,0.000000
25276,F4A9B95166FF93094F5B6A98D2D41B45,001C8AAB90AEE49F36FCAA7B4136C81A,VTS,1,NaN,2013-01-13 02:22:00,2013-01-13 02:32:00,1,600,2.89,...,2.366667,2,5.979824,35.878941,2013-01-13,2013-01-13,Night,3,0.000000,0.000000
4012,F4A9B95166FF93094F5B6A98D2D41B45,001C8AAB90AEE49F36FCAA7B4136C81A,VTS,1,NaN,2013-01-13 04:19:00,2013-01-13 04:41:00,1,1320,11.39,...,4.316667,4,15.443777,42.119392,2013-01-13,2013-01-13,Night,3,5948.979058,8.262471
4956,F4A9B95166FF93094F5B6A98D2D41B45,001C8AAB90AEE49F36FCAA7B4136C81A,VTS,1,NaN,2013-01-13 06:20:00,2013-01-13 06:30:00,1,600,3.64,...,6.333333,6,6.548440,39.290641,2013-01-13,2013-01-13,Day,0,13335.466406,18.521481
1531,F4A9B95166FF93094F5B6A98D2D41B45,001C8AAB90AEE49F36FCAA7B4136C81A,VTS,1,NaN,2013-01-13 06:40:00,2013-01-13 06:44:00,1,240,0.81,...,6.666667,6,0.767674,11.515105,2013-01-13,2013-01-13,Day,0,14206.429068,19.731151


In [239]:
sorted_train_data.keys()

Index(['medallion', 'hack_license', 'vendor_id', 'rate_code',
       'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
       'passenger_count', 'trip_time_in_secs', 'trip_distance',
       'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
       'dropoff_latitude', 'fare_amount', 'payment_type', 'surcharge',
       'mta_tax', 'tip_amount', 'tolls_amount', 'total_amount', 'pickup_day',
       'pickup_weekday', 'pickup_hour', 'pickup_minute', 'pickup_time',
       'dropoff_hour', 'distance', 'speed', 'pickup_date', 'dropoff_date',
       'time_inteval', 'day_interval', 'waiting_time', 'waiting_penalty'],
      dtype='object')

## Reinforcement Model

In [243]:
class TaxiWorld():
    def __init__(self):
        self.waiting_penalty = 5.00
        pass
                
    def getReward(self, row):
        return row['total_amount'] - row['waiting_time']/3600*self.waiting_penalty
    
    def set_waiting_penalty(self, penalty):
        self.waiting_penalty = penalty
        
    def getState(self):
        print('Current State')
        return self.s
    
    def setState(self, s):
        self.s = s
        
    def getStateSize(self, trip_and_fare):
        return len(trip_and_fare['zip_code'].unique())
    
    def getZipCodes(self, trip_and_fare):
        return self.zip_codes
    
    def getActionSize(self):
        return len(self.actions)

    def nextAction(self, s):
        print('Calculate Next Action based on state')
        
    def check_Q_table(self, s):
        if s not in self.Q:
            self.Q[s] = dict((action, 0.0) for action in self.actions)
            


In [244]:
env = TaxiWorld()

In [245]:
class TaxiRevenue:
    def __init__(self, env, penalty=5.0):
        self.env = env
        self.env.set_waiting_penalty(penalty)
        self.Q = dict()
    
    def generate_key(self, row):
        return str(row['pickup_latitude']) + '_' + str(row['pickup_longitude']) + '_'  + str(row['pickup_weekday']) + '_' + str(row['day_interval'])
    
    def get_tuples(self, key):
        tuples = key.split('_')
        return {'pickup_latitude': tuples[0],
                   'pickup_longitude': tuples[1],
                   'pickup_weekday': tuples[2],
                   'day_interval': tuples[3]}
    
    def build_Q_table(self, data):
        for index,row in data.T.iteritems():
            key = self.generate_key(row)
            reward = self.env.getReward(row)
            if key not in self.Q:
                self.Q[key] = [reward,1]
            else:
                self.Q[key] = [self.Q[key][0] +reward, self.Q[key][1] + 1]
                
    def greedy(self, s):
        return np.argmax(self.Q[s[0]]) 

    def epsilon_greed(self, epsilon, s):
        if np.random.rand() < epsilon:
            return np.random.randint(self.n_a)
        else:
            return self.greedy(s)
        
    def train(self, trip_and_fare):
        self.build_Q_table(trip_and_fare)
        return self.Q
        
    def test(self):
        return self.env.getState()

In [246]:
taxiRevenue = TaxiRevenue(env)
_Q = taxiRevenue.train(sorted_train_data)

In [247]:
_Q

{'40.645481_-73.77636_3_3': [65.3, 1],
 '40.730011_-73.983566_6_3': [13.7, 1],
 '40.74181_-73.993576_6_3': [35.13752908649161, 1],
 '40.786541_-73.942551_6_0': [-5.521481120098947, 1],
 '40.761356_-73.97953000000003_6_0': [-14.231151483324219, 1],
 '40.742538_-74.003914_6_3': [11.0, 1],
 '40.744495_-74.006416_6_3': [11.86833628040142, 1],
 '40.780106_-73.944664_6_0': [7.7, 1],
 '40.773781_-73.962051_6_0': [19.22670961200496, 1],
 '40.721611_-73.94641899999998_6_0': [4.235752763874016, 1],
 '40.714176_-73.95431500000002_6_0': [6.1674867520642485, 1],
 '40.717861_-73.985802_6_0': [5.84336127787516, 1],
 '40.734951_-73.990181_6_0': [8.930903416855893, 1],
 '40.782951_-73.98226899999997_6_1': [0.09076271260979851, 1],
 '40.78355_-73.981804_6_1': [0.21866853412991638, 1],
 '40.77541_-73.982422_6_1': [-1.8316636210283885, 1],
 '40.787308_-73.97159599999998_6_1': [5.791730514399829, 1],
 '40.758591_-73.99254599999998_6_1': [2.792933114709971, 1],
 '40.773392_-73.989136_6_1': [11.3367686015248

In [ ]:
# if key not in self.Q:
#                 self.Q[key] = tmp_dict
#             else:
#                 if action_key not in self.Q[key]:
#                     self.Q[key] = tmp_dict
#                 else:
#                     count = self.Q[key][action_key][0] + 1
#                     average_revenue = ((self.Q[key][action_key][0]*self.Q[key][action_key][1]) + row['total_amount'])/(count)
#                     self.Q[key][action_key] = [average_revenue, count]